<a href="https://colab.research.google.com/github/ColoAlfa/PracticaTesting/blob/master/TextClassification_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proyecto de classificación de texto hecho por Didac Colominas Abalde, estudiante de la EPS de la UDL en la rama de computación.



# **INTRODUCCIÓN AL PROYECTO**
Este cuaderno entrena un modelo de análisis de sentimientos para clasificar las reseñas de películas como positivas o negativas , según el texto de la reseña.Este tipo de analisis se llaman de clasificación binaria, o de dos clases.

In [2]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

#Comprobamos que version tenemos de tensorflow
print(tf.__version__)

2.4.1


# **DATASET**
Ahora procederemos a descargar y visualizar el DATASET. El set tiene tanto reviews positivas como negativas, i esta balanceado de manera que tiene las mismas positivas que negativas.



> Procedemos a descargar el DATASET



In [3]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

84131840/84125825 [==============================] - 3s 0us/step




> Ahora veremos la estructura del dataset un poco:



In [7]:
os.listdir(dataset_dir)


['imdbEr.txt', 'test', 'README', 'imdb.vocab', 'train']

In [8]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['pos',
 'unsup',
 'labeledBow.feat',
 'urls_unsup.txt',
 'urls_pos.txt',
 'unsupBow.feat',
 'urls_neg.txt',
 'neg']



Las reseñas positivas se encuentran en aclImdb/train/pos mientras que las negativas se encuentran en  aclImdb/train/neg. Probaremos a abrir cualquiera de las reviews.




In [10]:
sample_file = os.path.join(train_dir, 'pos/0_9.txt')
with open(sample_file) as f:
  print(f.read())

Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!


# **CONJUNTO DE DATOS**
A continuación prepararemos el formato de datos para poder entrenar. Para ello usaremos:


# tf.keras.preprocessing.text_dataset_from_directory:
Basicamente lo que hace es generar un dataset ya montado y listo para usarse. 
Para ello la estructura del directorio debe ser: 










```
main_directory/
...class_a/
......a_text_1.txt
......a_text_2.txt
...class_b/
......b_text_1.txt
......b_text_2.txt
```



Por lo tanto deberemos eliminar todos aquellos directorios que no sean aclImdb/train/pos y aclImdb/train/neg que son las dos classes possibles.

In [12]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

Ahora ya podemos usar text_dataset_from_directory para crear un dataset tratable. Se recomienda tener 3 subconjuntos de datos: de **ENTRENAMIENTO**, de **VALIDACIÓN** y de **PRUEBA**.

*   **Entrenamiento**: Subconjunto de datos que se usa para entrenar el modelo.
*   **Validación**: Subconjunto de datos que se usa para ajustar los hiperparametros. Seria como ajustar la linea divisoria.
*   **Prueba**: Subconjunto que se pasa cuando ya se han pasado el de entrenamiento y validacion,

Pero IMDB carece de set de validación, por lo que usaremos la función validation_split. Cogeremos los 25.000 de entrenamiento, y cogeremos el 80% para una nueva carpreta de entrenamiento llamada "training".




In [13]:
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='training', 
    seed=seed)


Found 25000 files belonging to 2 classes.
Using 20000 files for training.


Ahora de los restantes en la carpeta "train" cogeremos para validation. I ya definiremos los de prueba.

In [21]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', 
    batch_size=batch_size, 
    validation_split=0.2, 
    subset='validation', 
    seed=seed)

Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


In [20]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test', 
    batch_size=batch_size)

Found 25000 files belonging to 2 classes.


# **ESTANDARIZAR DATOS PARA EL ENTRENAMIENTO**
La estandaritzación se refiere a eliminar la puntuación o elementos HTML para simplificar el conjunto de datos. Hay dos terminos importantes:
*   **Tokenización**: Separar una frase por ejemplo en palabras (espacios).
*   **Vectorización**: convertir los tokens en números para poder introducirse a  la red neuronal.


Ahora vamos a por una parte minimizar el texto, y eliminar el codigo HTML.La función de abajo lo que hara es sustituir el "br />" por espacios " "

In [ ]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')